In [4]:
import pandas as pd

import extract_coverage
import report_util
import tables

coverage = extract_coverage.read_coverage_csv('../data/coverage.csv')
coverage.head()

,campaign_id,fuzzer,subject,time,covered_branches
0,0,BeDiv-Simple,Ant,0 days 00:00:00,59
1,0,BeDiv-Simple,Ant,0 days 00:00:10.810810810,66
2,0,BeDiv-Simple,Ant,0 days 00:00:21.621621621,68
3,0,BeDiv-Simple,Ant,0 days 00:00:32.432432432,68
4,0,BeDiv-Simple,Ant,0 days 00:00:43.243243243,68


In [ ]:
def create_rows(data, time, subject):
    data = data[data['time'] == time]
    data = data[data['subject'] == subject]
    fuzzers = sorted(data['fuzzer'].unique())
    rows = []
    values1 = data[data['fuzzer'] == 'Zeugma-Linked']['covered_branches']
    significance_level = 0.05 / (len(fuzzers) * (len(fuzzers) - 1) / 2)
    for fuzzer in fuzzers:
        values2 = data[data['fuzzer'] == fuzzer]['covered_branches']
        sig = 'color: red;' if report_util.mann_whitney(values1, values2) < significance_level else ''
        rows.append([time, subject, fuzzer, values2.median(), sig])
    return pd.DataFrame(rows, columns=['time', 'subject', 'fuzzer', 'median', 'sig'])


def create_sig_table(data, times):
    return pd.concat(
        [create_rows(data, time, subject) for time in times for subject in sorted(data['subject'].unique())])


times = [pd.to_timedelta(5, 'm'), pd.to_timedelta(3, 'h')]
coverage = extract_coverage.read_coverage_csv('../data/coverage.csv')
table = create_sig_table(coverage, times)
medians = tables.pivot(table, 'subject', 'fuzzer', 'median')
sigs = tables.pivot(table, 'subject', 'fuzzer', 'sig')
styler = tables.style_table(medians, precision=1, axis=0).apply(lambda _: sigs, axis=None)
display(styler)
print(styler.to_latex(multicol_align='c', hrules=True, multirow_align='t', convert_css=True))